In [134]:
import re

In [1]:
import pandas as pd
import numpy as np

from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver

from bs4 import BeautifulSoup 
import requests 

from tqdm import tnrange, tqdm_notebook as tqdm
import time
import random
from copy import deepcopy

import pickle

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://pokemondb.net/pokedex/all')

In [4]:
pokedex = []

In [5]:
for i in driver.find_element_by_id('pokedex').find_elements_by_css_selector('tr'):
    pokedex.append(i.text)

In [493]:
df0 = pd.DataFrame(pokedex)

In [30]:
df

,0,1,2,3,4,5,6,7,8,9
0,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
1,001 Bulbasaur GRASS,POISON 318 45 49 49 65 65 45,None,None,None,None,None,None,None,None
2,002 Ivysaur GRASS,POISON 405 60 62 63 80 80 60,None,None,None,None,None,None,None,None
3,003 Venusaur GRASS,POISON 525 80 82 83 100 100 80,None,None,None,None,None,None,None,None
4,003 Venusaur,Mega Venusaur GRASS,POISON 625 80 100 123 122 120 80,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
1024,888 Zacian,Hero of Many Battles FAIRY,670 92 130 115 80 115 138,None,None,None,None,None,None,None
1025,889 Zamazenta,Crowned Shield FIGHTING,STEEL 720 92 130 145 80 145 128,None,None,None,None,None,None,None
1026,889 Zamazenta,Hero of Many Battles FIGHTING,670 92 130 115 80 115 138,None,None,None,None,None,None,None
1027,890 Eternatus POISON,DRAGON 690 140 85 95 145 95 130,None,None,None,None,None,None,None,None


In [9]:
df = df[0].str.split('\n', expand=True)

In [11]:
name = df[0].str.split(' ', expand = True)

In [14]:
name.head()

,0,1,2,3
0,#,None,None,None
1,001,Bulbasaur,GRASS,None
2,002,Ivysaur,GRASS,None
3,003,Venusaur,GRASS,None
4,003,Venusaur,None,None


In [12]:
types = name[2].unique().tolist() 

In [15]:
types2 = name[3].unique().tolist() 

In [17]:
types2

[None, 'PSYCHIC', 'NORMAL', 'ELECTRIC', 'GRASS', 'WATER']

In [18]:
for i in types2:
    types.append(i)

In [20]:
types = list(set(types))

In [27]:
types.remove(None)

In [28]:
final_types = []
for i in types:
    if i.isupper() == True:
        final_types.append(i)

In [29]:
final_types

['NORMAL',
 'FIGHTING',
 'PSYCHIC',
 'FIRE',
 'GHOST',
 'FAIRY',
 'BUG',
 'STEEL',
 'FLYING',
 'WATER',
 'GRASS',
 'ROCK',
 'DARK',
 'DRAGON',
 'ELECTRIC',
 'GROUND',
 'POISON',
 'ICE']

In [44]:
column_names= df.loc[0].tolist()

In [47]:
df.columns = column_names

In [50]:
df.drop(0, inplace=True)

In [54]:
df['#']

1        001 Bulbasaur GRASS
2          002 Ivysaur GRASS
3         003 Venusaur GRASS
4               003 Venusaur
5        004 Charmander FIRE
                ...         
1024              888 Zacian
1025           889 Zamazenta
1026           889 Zamazenta
1027    890 Eternatus POISON
1028           890 Eternatus
Name: #, Length: 1028, dtype: object

## Just get the first 3 in column

In [79]:
df['pokemon_no'] = [int(x[:3]) for x in df['#']]

In [80]:
df

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,pokemon_no
1,001 Bulbasaur GRASS,POISON 318 45 49 49 65 65 45,None,None,None,None,None,None,None,None,1
2,002 Ivysaur GRASS,POISON 405 60 62 63 80 80 60,None,None,None,None,None,None,None,None,2
3,003 Venusaur GRASS,POISON 525 80 82 83 100 100 80,None,None,None,None,None,None,None,None,3
4,003 Venusaur,Mega Venusaur GRASS,POISON 625 80 100 123 122 120 80,None,None,None,None,None,None,None,3
5,004 Charmander FIRE,309 39 52 43 60 50 65,None,None,None,None,None,None,None,None,4
...,...,...,...,...,...,...,...,...,...,...,...
1024,888 Zacian,Hero of Many Battles FAIRY,670 92 130 115 80 115 138,None,None,None,None,None,None,None,888
1025,889 Zamazenta,Crowned Shield FIGHTING,STEEL 720 92 130 145 80 145 128,None,None,None,None,None,None,None,889
1026,889 Zamazenta,Hero of Many Battles FIGHTING,670 92 130 115 80 115 138,None,None,None,None,None,None,None,889
1027,890 Eternatus POISON,DRAGON 690 140 85 95 145 95 130,None,None,None,None,None,None,None,None,890


## REGEX

### Pokemon Number

In [135]:
df['#'][700]

'601 Klinklang STEEL'

In [136]:
df['#']

1        001 Bulbasaur GRASS
2          002 Ivysaur GRASS
3         003 Venusaur GRASS
4               003 Venusaur
5        004 Charmander FIRE
                ...         
1024              888 Zacian
1025           889 Zamazenta
1026           889 Zamazenta
1027    890 Eternatus POISON
1028           890 Eternatus
Name: #, Length: 1028, dtype: object

In [146]:
pattern = re.compile(r'\d{3}')

In [147]:
matches = pattern.finditer(df['#'][700])

In [148]:
for match in matches:
    print(match)

<re.Match object; span=(0, 3), match='601'>


In [82]:
# df['#'].str.extract(r')

1        001 Bulbasaur GRASS
2          002 Ivysaur GRASS
3         003 Venusaur GRASS
4               003 Venusaur
5        004 Charmander FIRE
                ...         
1024              888 Zacian
1025           889 Zamazenta
1026           889 Zamazenta
1027    890 Eternatus POISON
1028           890 Eternatus
Name: #, Length: 1028, dtype: object

### Pokemon number w/ regex

In [86]:
df0 = pd.DataFrame(pokedex)

In [123]:
df0[0][1]

'001 Bulbasaur GRASS\nPOISON 318 45 49 49 65 65 45'

In [124]:
df0[0][100]

'077 Ponyta FIRE\n410 50 85 55 65 65 90'

In [125]:
df0[0][250]

'208 Steelix STEEL\nGROUND 510 75 85 200 55 65 30'

In [174]:
text_to_search = df0[0][1]

text_to_search

pattern = re.compile(r'^[0-9]{3}')

matches = pattern.finditer(text_to_search)

for match in matches:
    print(match)

In [200]:
df0[0].str.extract(r'(^[0-9]{3})')

,0
0,NaN
1,001
2,002
3,003
4,003
...,...
1024,888
1025,889
1026,889
1027,890


In [203]:
df0['pokemon_index'] = df0[0].str.extract(r'(^[0-9]{3})')

### Pokemon Name

In [282]:
df0.rename(columns= {'0_x':0, '0_y':'type'}, inplace=True)

In [283]:
df0

,0,pokemon_index,type
0,#\nName\nType\nTotal\nHP\nAttack\nDefense\nSp....,NaN,HP
1,001 Bulbasaur GRASS\nPOISON 318 45 49 49 65 65 45,001,"GRASS, POISON"
2,002 Ivysaur GRASS\nPOISON 405 60 62 63 80 80 60,002,"GRASS, POISON"
3,003 Venusaur GRASS\nPOISON 525 80 82 83 100 10...,003,"GRASS, POISON"
4,003 Venusaur\nMega Venusaur GRASS\nPOISON 625 ...,003,"GRASS, POISON"
...,...,...,...
1024,888 Zacian\nHero of Many Battles FAIRY\n670 92...,888,FAIRY
1025,889 Zamazenta\nCrowned Shield FIGHTING\nSTEEL ...,889,"FIGHTING, STEEL"
1026,889 Zamazenta\nHero of Many Battles FIGHTING\n...,889,FIGHTING
1027,890 Eternatus POISON\nDRAGON 690 140 85 95 145...,890,"POISON, DRAGON"


In [300]:
text_to_search = df0[0][3]

In [301]:
text_to_search2 = df0[0][4]

In [302]:
text_to_search

'003 Venusaur GRASS\nPOISON 525 80 82 83 100 100 80'

In [303]:
text_to_search2

'003 Venusaur\nMega Venusaur GRASS\nPOISON 625 80 100 123 122 120 80'

In [298]:
df0[0][3]

'003 Venusaur GRASS\nPOISON 525 80 82 83 100 100 80'

In [299]:
df0[0][4]

'003 Venusaur\nMega Venusaur GRASS\nPOISON 625 80 100 123 122 120 80'

### Double Names

In [315]:
test = df0[0].str.split('\n', expand=True)

In [319]:
test = test[~test[2].isna()]

In [371]:
test['type_1'] = test[1].str.rsplit(" ", 1).str[-1]

/Users/johnrick/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [374]:
test['secondary_name'] = test[1].replace(to_replace=r'\b'+test['type_1']+r'\b', value='',regex=True)

/Users/johnrick/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [380]:
test_2= test[2].str.split(' ', expand=True)

In [384]:
only_one_type = test_2[test_2[7].isna()]

In [387]:
two_types = test_2[~test_2[7].isna()]

In [390]:
test.head()

,0,1,2,3,4,5,6,7,8,9,type_1,secondary_name
0,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Name,
4,003 Venusaur,Mega Venusaur GRASS,POISON 625 80 100 123 122 120 80,None,None,None,None,None,None,None,GRASS,Mega Venusaur
8,006 Charizard,Mega Charizard X FIRE,DRAGON 634 78 130 111 130 85 100,None,None,None,None,None,None,None,FIRE,Mega Charizard X
9,006 Charizard,Mega Charizard Y FIRE,FLYING 634 78 104 78 159 115 100,None,None,None,None,None,None,None,FIRE,Mega Charizard Y
13,009 Blastoise,Mega Blastoise WATER,630 79 103 120 135 115 78,None,None,None,None,None,None,None,WATER,Mega Blastoise


In [425]:
test.loc[4]

0                                     003 Venusaur
1                              Mega Venusaur GRASS
2                 POISON 625 80 100 123 122 120 80
total                                         None
hp                                            None
attack                                        None
defense                                       None
sp_atk                                        None
sp_def                                        None
speed                                         None
type_1                                       GRASS
secondary_name                      Mega Venusaur 
Name: 4, dtype: object

In [426]:
two_types.loc[4]

type_2     POISON
total         625
hp             80
attack        100
defense       123
sp_atk        122
sp_def        120
speed          80
Name: 4, dtype: object

In [430]:
test.merge(two_types,how= 'outer',left_index=True, right_index=True)

,0,1,2,total_x,hp_x,attack_x,defense_x,sp_atk_x,sp_def_x,speed_x,type_1,secondary_name,type_2,total_y,hp_y,attack_y,defense_y,sp_atk_y,sp_def_y,speed_y
0,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Name,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,003 Venusaur,Mega Venusaur GRASS,POISON 625 80 100 123 122 120 80,None,None,None,None,None,None,None,GRASS,Mega Venusaur,POISON,625,80,100,123,122,120,80
8,006 Charizard,Mega Charizard X FIRE,DRAGON 634 78 130 111 130 85 100,None,None,None,None,None,None,None,FIRE,Mega Charizard X,DRAGON,634,78,130,111,130,85,100
9,006 Charizard,Mega Charizard Y FIRE,FLYING 634 78 104 78 159 115 100,None,None,None,None,None,None,None,FIRE,Mega Charizard Y,FLYING,634,78,104,78,159,115,100
13,009 Blastoise,Mega Blastoise WATER,630 79 103 120 135 115 78,None,None,None,None,None,None,None,WATER,Mega Blastoise,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,888 Zacian,Crowned Sword FAIRY,STEEL 720 92 170 115 80 115 148,None,None,None,None,None,None,None,FAIRY,Crowned Sword,STEEL,720,92,170,115,80,115,148
1024,888 Zacian,Hero of Many Battles FAIRY,670 92 130 115 80 115 138,None,None,None,None,None,None,None,FAIRY,Hero of Many Battles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1025,889 Zamazenta,Crowned Shield FIGHTING,STEEL 720 92 130 145 80 145 128,None,None,None,None,None,None,None,FIGHTING,Crowned Shield,STEEL,720,92,130,145,80,145,128
1026,889 Zamazenta,Hero of Many Battles FIGHTING,670 92 130 115 80 115 138,None,None,None,None,None,None,None,FIGHTING,Hero of Many Battles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [416]:
only_one_type.merge(test, how='outer',left_index=True,right_index=True)

,total_x,hp_x,attack_x,defense_x,sp_atk_x,sp_def_x,speed_x,7,0,1,2,total_y,hp_y,attack_y,defense_y,sp_atk_y,sp_def_y,speed_y,type_1,secondary_name
0,Type,None,None,None,None,None,None,None,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Name,
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,003 Venusaur,Mega Venusaur GRASS,POISON 625 80 100 123 122 120 80,None,None,None,None,None,None,None,GRASS,Mega Venusaur
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,006 Charizard,Mega Charizard X FIRE,DRAGON 634 78 130 111 130 85 100,None,None,None,None,None,None,None,FIRE,Mega Charizard X
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,006 Charizard,Mega Charizard Y FIRE,FLYING 634 78 104 78 159 115 100,None,None,None,None,None,None,None,FIRE,Mega Charizard Y
13,630,79,103,120,135,115,78,None,009 Blastoise,Mega Blastoise WATER,630 79 103 120 135 115 78,None,None,None,None,None,None,None,WATER,Mega Blastoise
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,888 Zacian,Crowned Sword FAIRY,STEEL 720 92 170 115 80 115 148,None,None,None,None,None,None,None,FAIRY,Crowned Sword
1024,670,92,130,115,80,115,138,None,888 Zacian,Hero of Many Battles FAIRY,670 92 130 115 80 115 138,None,None,None,None,None,None,None,FAIRY,Hero of Many Battles
1025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,889 Zamazenta,Crowned Shield FIGHTING,STEEL 720 92 130 145 80 145 128,None,None,None,None,None,None,None,FIGHTING,Crowned Shield
1026,670,92,130,115,80,115,138,None,889 Zamazenta,Hero of Many Battles FIGHTING,670 92 130 115 80 115 138,None,None,None,None,None,None,None,FIGHTING,Hero of Many Battles


In [434]:
test.columns.to_list()

[0,
 1,
 2,
 'total',
 'hp',
 'attack',
 'defense',
 'sp_atk',
 'sp_def',
 'speed',
 'type_1',
 'secondary_name']

In [439]:
name2 = test[[0,
      'secondary_name',
      'type_1']]

In [448]:
name = name.merge(name2, how='left',left_index=True, right_index=True)

In [450]:
name.columns.to_list()

['0_x', 1, 2, 3, '0_y', 'secondary_name', 'type_1']

In [451]:
name.head()

,0_x,1,2,3,0_y,secondary_name,type_1
0,#,None,None,None,#,,Name
1,001,Bulbasaur,GRASS,None,NaN,NaN,NaN
2,002,Ivysaur,GRASS,None,NaN,NaN,NaN
3,003,Venusaur,GRASS,None,NaN,NaN,NaN
4,003,Venusaur,None,None,003 Venusaur,Mega Venusaur,GRASS


In [453]:
name= name[['0_x', 1, 2, 3, 'secondary_name', 'type_1']]

In [456]:
name[~name[3].isna()]

,0_x,1,2,3,secondary_name,type_1
156,122,Mr.,Mime,PSYCHIC,NaN,NaN
520,439,Mime,Jr.,PSYCHIC,NaN,NaN
899,772,Type:,Null,NORMAL,NaN,NaN
913,785,Tapu,Koko,ELECTRIC,NaN,NaN
914,786,Tapu,Lele,PSYCHIC,NaN,NaN
915,787,Tapu,Bulu,GRASS,NaN,NaN
916,788,Tapu,Fini,WATER,NaN,NaN
998,866,Mr.,Rime,PSYCHIC,NaN,NaN


In [444]:
only_one_type.head()

,total,hp,attack,defense,sp_atk,sp_def,speed,7
0,Type,None,None,None,None,None,None,None
13,630,79,103,120,135,115,78,None
34,430,45,80,50,75,60,120,None
50,299,38,41,40,50,65,65,None
69,290,40,35,35,50,40,90,None


In [445]:
two_types.head()

,type_2,total,hp,attack,defense,sp_atk,sp_def,speed
4,POISON,625,80,100,123,122,120,80
8,DRAGON,634,78,130,111,130,85,100
9,FLYING,634,78,104,78,159,115,100
20,POISON,495,65,150,40,15,80,145
24,FLYING,579,83,80,80,135,80,121


In [396]:
only_one_type.rename(columns = {0:'total',1:'hp',2:'attack',3:'defense',4:'sp_atk', 5:'sp_def', 6:'speed'},inplace=True)

/Users/johnrick/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [398]:
only_one_type[~only_one_type[7].isna()]

,total,hp,attack,defense,sp_atk,sp_def,speed,7


In [402]:
two_types.rename(columns = {0:'type_2',1:'total',2:'hp',3:'attack',4:'defense',5:'sp_atk', 6:'sp_def', 7:'speed'}, inplace=True)

/Users/johnrick/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [405]:
test.rename(columns = {3:'total',4:'hp',5:'attack',6:'defense',7:'sp_atk', 8:'sp_def', 9:'speed'},inplace=True)

In [406]:
test.head()

,0,1,2,total,hp,attack,defense,sp_atk,sp_def,speed,type_1,secondary_name
0,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Name,
4,003 Venusaur,Mega Venusaur GRASS,POISON 625 80 100 123 122 120 80,None,None,None,None,None,None,None,GRASS,Mega Venusaur
8,006 Charizard,Mega Charizard X FIRE,DRAGON 634 78 130 111 130 85 100,None,None,None,None,None,None,None,FIRE,Mega Charizard X
9,006 Charizard,Mega Charizard Y FIRE,FLYING 634 78 104 78 159 115 100,None,None,None,None,None,None,None,FIRE,Mega Charizard Y
13,009 Blastoise,Mega Blastoise WATER,630 79 103 120 135 115 78,None,None,None,None,None,None,None,WATER,Mega Blastoise


In [394]:
df0.merge()

,0,pokemon_index,type
0,#\nName\nType\nTotal\nHP\nAttack\nDefense\nSp....,NaN,HP
1,001 Bulbasaur GRASS\nPOISON 318 45 49 49 65 65 45,001,"GRASS, POISON"
2,002 Ivysaur GRASS\nPOISON 405 60 62 63 80 80 60,002,"GRASS, POISON"
3,003 Venusaur GRASS\nPOISON 525 80 82 83 100 10...,003,"GRASS, POISON"
4,003 Venusaur\nMega Venusaur GRASS\nPOISON 625 ...,003,"GRASS, POISON"
...,...,...,...
1024,888 Zacian\nHero of Many Battles FAIRY\n670 92...,888,FAIRY
1025,889 Zamazenta\nCrowned Shield FIGHTING\nSTEEL ...,889,"FIGHTING, STEEL"
1026,889 Zamazenta\nHero of Many Battles FIGHTING\n...,889,FIGHTING
1027,890 Eternatus POISON\nDRAGON 690 140 85 95 145...,890,"POISON, DRAGON"


In [367]:
test[1].

0                                Name
4                 Mega Venusaur GRASS
8               Mega Charizard X FIRE
9               Mega Charizard Y FIRE
13               Mega Blastoise WATER
                    ...              
1023              Crowned Sword FAIRY
1024       Hero of Many Battles FAIRY
1025          Crowned Shield FIGHTING
1026    Hero of Many Battles FIGHTING
1028                 Eternamax POISON
Name: 1, Length: 166, dtype: object

In [323]:
#get rid of capitalized with more than one
test[1]

0                                Name
4                 Mega Venusaur GRASS
8               Mega Charizard X FIRE
9               Mega Charizard Y FIRE
13               Mega Blastoise WATER
                    ...              
1023              Crowned Sword FAIRY
1024       Hero of Many Battles FAIRY
1025          Crowned Shield FIGHTING
1026    Hero of Many Battles FIGHTING
1028                 Eternamax POISON
Name: 1, Length: 166, dtype: object

In [334]:
text_2_search = test[1][4]

In [335]:
text_2_search2 = test[1][8]

In [336]:
text_2_search

'Mega Venusaur GRASS'

In [338]:
text_2_search2

'Mega Charizard X FIRE'

In [339]:
pattern = re.compile(r'[A-Z]{2,}')

In [342]:
matches = pattern.finditer(text_2_search2)

In [343]:
for match in matches:
    print(match)

<re.Match object; span=(17, 21), match='FIRE'>


In [348]:
test[1].str.extract(r'([A-Z]{2,})')

TypeError: extract() takes from 2 to 4 positional arguments but 12 were given

In [365]:
test[1].str.extractall(r'(?:([A-Z]{2,}))')

,,0
,match,
4,0,GRASS
8,0,FIRE
9,0,FIRE
13,0,WATER
20,0,BUG
...,...,...
1023,0,FAIRY
1024,0,FAIRY
1025,0,FIGHTING


### Pokemon Type

In [218]:
text_to_search = df0[0][4]

In [220]:
text_to_search2 = df0[0][1]

In [219]:
text_to_search

'003 Venusaur\nMega Venusaur GRASS\nPOISON 625 80 100 123 122 120 80'

In [221]:
text_to_search2

'001 Bulbasaur GRASS\nPOISON 318 45 49 49 65 65 45'

In [229]:
pattern = re.compile(r'[A-Z]{2,}')

In [232]:
matches = pattern.finditer(text_to_search2)

In [233]:
for match in matches:
    print(match)

<re.Match object; span=(14, 19), match='GRASS'>
<re.Match object; span=(20, 26), match='POISON'>


In [245]:
column_names = df0[0].str.split('\n')[0]

In [246]:
column_names

['#',
 'Name',
 'Type',
 'Total',
 'HP',
 'Attack',
 'Defense',
 'Sp. Atk',
 'Sp. Def',
 'Speed']

In [253]:
#to extract from dataframe
s = df0[0].str.extractall(r'([A-Z]{2,})')

In [264]:
s

0
     match          
0    0            HP
1    0         GRASS
     1        POISON
2    0         GRASS
     1        POISON
...              ...
1026 0      FIGHTING
1027 0        POISON
     1        DRAGON
1028 0        POISON
     1        DRAGON

[1571 rows x 1 columns]

In [268]:
s = s[0].groupby(level=0).apply(', '.join)

In [272]:
type(s)

pandas.core.series.Series

In [276]:
df0 = df0.merge(s, how = 'inner', left_index=True, right_index=True)

In [277]:
df0

,0_x,pokemon_index,0_y
0,#\nName\nType\nTotal\nHP\nAttack\nDefense\nSp....,NaN,HP
1,001 Bulbasaur GRASS\nPOISON 318 45 49 49 65 65 45,001,"GRASS, POISON"
2,002 Ivysaur GRASS\nPOISON 405 60 62 63 80 80 60,002,"GRASS, POISON"
3,003 Venusaur GRASS\nPOISON 525 80 82 83 100 10...,003,"GRASS, POISON"
4,003 Venusaur\nMega Venusaur GRASS\nPOISON 625 ...,003,"GRASS, POISON"
...,...,...,...
1024,888 Zacian\nHero of Many Battles FAIRY\n670 92...,888,FAIRY
1025,889 Zamazenta\nCrowned Shield FIGHTING\nSTEEL ...,889,"FIGHTING, STEEL"
1026,889 Zamazenta\nHero of Many Battles FIGHTING\n...,889,FIGHTING
1027,890 Eternatus POISON\nDRAGON 690 140 85 95 145...,890,"POISON, DRAGON"


In [204]:
df0

,0,pokemon_index
0,#\nName\nType\nTotal\nHP\nAttack\nDefense\nSp....,NaN
1,001 Bulbasaur GRASS\nPOISON 318 45 49 49 65 65 45,001
2,002 Ivysaur GRASS\nPOISON 405 60 62 63 80 80 60,002
3,003 Venusaur GRASS\nPOISON 525 80 82 83 100 10...,003
4,003 Venusaur\nMega Venusaur GRASS\nPOISON 625 ...,003
...,...,...
1024,888 Zacian\nHero of Many Battles FAIRY\n670 92...,888
1025,889 Zamazenta\nCrowned Shield FIGHTING\nSTEEL ...,889
1026,889 Zamazenta\nHero of Many Battles FIGHTING\n...,889
1027,890 Eternatus POISON\nDRAGON 690 140 85 95 145...,890


In [98]:
df0[0][1].extract(pattern)

AttributeError: 'str' object has no attribute 'extract'

,0
0,#\nName\nType\nTotal\nHP\nAttack\nDefense\nSp....
1,001 Bulbasaur GRASS\nPOISON 318 45 49 49 65 65 45
2,002 Ivysaur GRASS\nPOISON 405 60 62 63 80 80 60
3,003 Venusaur GRASS\nPOISON 525 80 82 83 100 10...
4,003 Venusaur\nMega Venusaur GRASS\nPOISON 625 ...
...,...
1024,888 Zacian\nHero of Many Battles FAIRY\n670 92...
1025,889 Zamazenta\nCrowned Shield FIGHTING\nSTEEL ...
1026,889 Zamazenta\nHero of Many Battles FIGHTING\n...
1027,890 Eternatus POISON\nDRAGON 690 140 85 95 145...


# REGEX 2

In [ ]:
# df['#'].str.extract(r')

In [486]:
df0[0].to_list()[867:]

['744 Rockruff ROCK\n280 45 65 40 30 40 60',
 '744 Rockruff\nOwn Tempo Rockruff ROCK\n280 45 65 40 30 40 60',
 '745 Lycanroc\nMidday Form ROCK\n487 75 115 65 55 65 112',
 '745 Lycanroc\nMidnight Form ROCK\n487 85 115 75 55 75 82',
 '745 Lycanroc\nDusk Form ROCK\n487 75 117 65 55 65 110',
 '746 Wishiwashi\nSolo Form WATER\n175 45 20 20 25 25 40',
 '746 Wishiwashi\nSchool Form WATER\n620 45 140 130 140 135 30',
 '747 Mareanie POISON\nWATER 305 50 53 62 43 52 45',
 '748 Toxapex POISON\nWATER 495 50 63 152 53 142 35',
 '749 Mudbray GROUND\n385 70 100 70 45 55 45',
 '750 Mudsdale GROUND\n500 100 125 100 55 85 35',
 '751 Dewpider WATER\nBUG 269 38 40 52 40 72 27',
 '752 Araquanid WATER\nBUG 454 68 70 92 50 132 42',
 '753 Fomantis GRASS\n250 40 55 35 50 35 35',
 '754 Lurantis GRASS\n480 70 105 90 80 90 45',
 '755 Morelull GRASS\nFAIRY 285 40 35 55 65 75 15',
 '756 Shiinotic GRASS\nFAIRY 405 60 45 80 90 100 30',
 '757 Salandit POISON\nFIRE 320 48 44 40 71 40 77',
 '758 Salazzle POISON\nFIRE 48

## Name

In [494]:
df0['name'] = df0[0].str.extract(r'([A-Z][a-z.-]+\s?)')

## Secondary Name

In [500]:
df0[0].str.extract(r'([/n][A-Z1-9][a-z.-1-9%]+\s?[A-Za-z.-]+\s?[A-Za-z.-]+?\s)')

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1024,NaN
1025,NaN
1026,NaN
1027,NaN


In [519]:
df0['test']= df0[0].str.split('\n',expand=True)[1]

In [523]:
df0['secondary_name']= df0['test'].str.extract(r'([A-Z1-9][a-z.-1-9%]+\s?[A-Za-z.-]+\s?[A-Za-z.-]+?\s)')

In [530]:
df0.head()

,0,name,test,secondary_name
0,#\nName\nType\nTotal\nHP\nAttack\nDefense\nSp....,Name\n,Name,NaN
1,001 Bulbasaur GRASS\nPOISON 318 45 49 49 65 65 45,Bulbasaur,POISON 318 45 49 49 65 65 45,NaN
2,002 Ivysaur GRASS\nPOISON 405 60 62 63 80 80 60,Ivysaur,POISON 405 60 62 63 80 80 60,NaN
3,003 Venusaur GRASS\nPOISON 525 80 82 83 100 10...,Venusaur,POISON 525 80 82 83 100 100 80,NaN
4,003 Venusaur\nMega Venusaur GRASS\nPOISON 625 ...,Venusaur\n,Mega Venusaur GRASS,Mega Venusaur


In [535]:
df0['pokedex_number'] = df0[0].str.extract(r'([0-9]{3})')

In [545]:
df0.drop(columns=['test'],inplace=True)

In [550]:
df0['test']= df0[0].str.split(' ',1,expand=True)[1]

In [585]:
test = df0['test'].str.findall(r'([1-9][0-9]?[0-9]?[0-9]?)')

In [586]:
test1 = test.to_list()

In [607]:
test2 = pd.DataFrame(test1)

In [611]:
test2.rename(columns = {0:'total',1:'hp',2:'attack',3:'defense',4:'sp_attack',5:'sp_defense',6:'speed'}, inplace=True)

In [616]:
df0 = df0.merge(test2, how='inner', left_index=True, right_index=True)

In [620]:
df0 = df0.drop(columns = ['test','powers',0])

In [624]:
df0.drop(0,inplace=True)

In [626]:
df0.reset_index()

,index,name,secondary_name,pokedex_number,total,hp,attack,defense,sp_attack,sp_defense,speed
0,1,Bulbasaur,NaN,001,318,45,49,49,65,65,45
1,2,Ivysaur,NaN,002,405,60,62,63,80,80,60
2,3,Venusaur,NaN,003,525,80,82,83,100,100,80
3,4,Venusaur\n,Mega Venusaur,003,625,80,100,123,122,120,80
4,5,Charmander,NaN,004,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...
1023,1024,Zacian\n,Hero of Many,888,670,92,130,115,80,115,138
1024,1025,Zamazenta\n,Crowned Shield,889,720,92,130,145,80,145,128
1025,1026,Zamazenta\n,Hero of Many,889,670,92,130,115,80,115,138
1026,1027,Eternatus,NaN,890,690,140,85,95,145,95,130


In [627]:
df0

,name,secondary_name,pokedex_number,total,hp,attack,defense,sp_attack,sp_defense,speed
1,Bulbasaur,NaN,001,318,45,49,49,65,65,45
2,Ivysaur,NaN,002,405,60,62,63,80,80,60
3,Venusaur,NaN,003,525,80,82,83,100,100,80
4,Venusaur\n,Mega Venusaur,003,625,80,100,123,122,120,80
5,Charmander,NaN,004,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...
1024,Zacian\n,Hero of Many,888,670,92,130,115,80,115,138
1025,Zamazenta\n,Crowned Shield,889,720,92,130,145,80,145,128
1026,Zamazenta\n,Hero of Many,889,670,92,130,115,80,115,138
1027,Eternatus,NaN,890,690,140,85,95,145,95,130


In [613]:
test2

,total,hp,attack,defense,sp_attack,sp_defense,speed
0,None,None,None,None,None,None,None
1,318,45,49,49,65,65,45
2,405,60,62,63,80,80,60
3,525,80,82,83,100,100,80
4,625,80,100,123,122,120,80
...,...,...,...,...,...,...,...
1024,670,92,130,115,80,115,138
1025,720,92,130,145,80,145,128
1026,670,92,130,115,80,115,138
1027,690,140,85,95,145,95,130


In [603]:
test1[835].remove('10')

In [606]:
test1[280]

['515', '85', '80', '90', '105', '95', '60']

In [575]:
test.groupby(['match'])

In [571]:
test.groupby(by = ).to_list()

SyntaxError: invalid syntax (<ipython-input-571-e9896bddfab2>, line 1)

In [557]:
df0.head()

,0,name,secondary_name,pokedex_number,test,powers
0,#\nName\nType\nTotal\nHP\nAttack\nDefense\nSp....,Name\n,NaN,NaN,Atk\nSp. Def\nSpeed,[]
1,001 Bulbasaur GRASS\nPOISON 318 45 49 49 65 65 45,Bulbasaur,NaN,001,Bulbasaur GRASS\nPOISON 318 45 49 49 65 65 45,"[318, 45, 49, 49, 65, 65, 45]"
2,002 Ivysaur GRASS\nPOISON 405 60 62 63 80 80 60,Ivysaur,NaN,002,Ivysaur GRASS\nPOISON 405 60 62 63 80 80 60,"[405, 60, 62, 63, 80, 80, 60]"
3,003 Venusaur GRASS\nPOISON 525 80 82 83 100 10...,Venusaur,NaN,003,Venusaur GRASS\nPOISON 525 80 82 83 100 100 80,"[525, 80, 82, 83, 100, 100, 80]"
4,003 Venusaur\nMega Venusaur GRASS\nPOISON 625 ...,Venusaur\n,Mega Venusaur,003,Venusaur\nMega Venusaur GRASS\nPOISON 625 80 1...,"[625, 80, 100, 123, 122, 120, 80]"


In [560]:
df0['powers'][1]

['318', '45', '49', '49', '65', '65', '45']

In [ ]:
aa

In [39]:
df0 = df0[0].str.split(' ', expand = True)

In [40]:
df0

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,#\nName\nType\nTotal\nHP\nAttack\nDefense\nSp.,Atk\nSp.,Def\nSpeed,None,None,None,None,None,None,None,None,None,None
1,001,Bulbasaur,GRASS\nPOISON,318,45,49,49,65,65,45,None,None,None
2,002,Ivysaur,GRASS\nPOISON,405,60,62,63,80,80,60,None,None,None
3,003,Venusaur,GRASS\nPOISON,525,80,82,83,100,100,80,None,None,None
4,003,Venusaur\nMega,Venusaur,GRASS\nPOISON,625,80,100,123,122,120,80,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,888,Zacian\nHero,of,Many,Battles,FAIRY\n670,92,130,115,80,115,138,None
1025,889,Zamazenta\nCrowned,Shield,FIGHTING\nSTEEL,720,92,130,145,80,145,128,None,None
1026,889,Zamazenta\nHero,of,Many,Battles,FIGHTING\n670,92,130,115,80,115,138,None
1027,890,Eternatus,POISON\nDRAGON,690,140,85,95,145,95,130,None,None,None
